# Database class

In [3]:
import pymysql
from PIL import Image
import numpy as np
import cv2
from pathlib import Path
import re
from time import gmtime, strftime
import datetime

In [4]:
"""
==================================================
Class database : To handle connection and quarries 
 * methods:-
 * createTable : To create tables with at least 
                  one column in it. 
 * addColumn   : To add new columns or column to a 
                 specific table you created before. 
 * dropTable   : To delete table from a database. 
 * dropColumn  : To delete column. 
 * alterTable  : To modify a table. 
 * insert      : To insert new data
 * deleteAll   : To delete all data.
 * delete      : To delete based on condition. 
 * readAll     : To select all data.
 * read        : To read based on condition.
 
 Author : Zynab Smaan
 =================================================
 
""" 

'\n==================================================\nClass database : To handle connection and quarries \n * methods:-\n * createTable : To create tables with at least \n                  one column in it. \n * addColumn   : To add new columns or column to a \n                 specific table you created before. \n * dropTable   : To delete table from a database. \n * dropColumn  : To delete column. \n * alterTable  : To modify a table. \n * insert      : To insert new data\n * deleteAll   : To delete all data.\n * delete      : To delete based on condition. \n * readAll     : To select all data.\n * read        : To read based on condition.\n \n Author : Zynab Smaan\n =================================================\n \n'

In [5]:
class database :
    def __init__(self, database_name) :
        #create connection to the database
        self.IP = '127.0.0.1'
        self.DB_NAME = database_name
        self.PASSWORD = ''
        self.USER_NAME = 'root'
        
        self.DB = pymysql.connect(self.IP, self.USER_NAME, self.PASSWORD, self.DB_NAME)
        self.cursor = self.DB.cursor()
     
    
    def createTable(self, tableName, columnName, columnType, constraint='') :
        """
         This fuction is used to create table :
         * table_name : Name of the table.
         * column_name: Basic column or any column you want.
         * column_type: The type of the column.
         * constraint : Like not null  string, defult:no constraint.
          ** output   : Table with just one column.
        """
        try :
            sql = "create table " + tableName +"( " +columnName + " " + columnType + " " + constraint + " )"  #sql querry
            self.cursor.execute(sql) 
            print(tableName +" is created sucessfully!")
        except Exception as e :
            print('Error is: ' + str(e))
            
            
    def addColumns(self, tableName, columnNames, columnTypes) :
        """
        To add new_columns 
         * TableName   : The name of the table you want to modify.
         * ColumnNames : List of the new column names you want to add.
         * ColumnTypes : List of types of the columns you added in the same order.
         ** Output      : Table with this new columns. 
        """        
        try :
            for i in range(0, len(columnNames)) :
                sql = 'ALTER TABLE '+ tableName+ ' ADD ' + columnNames[i]+ " " + columnTypes[i]
                self.cursor.execute(sql)
            print("table is updated successfully!")    
        except Exception as e :
            print("Error is: " + str(e))

            
    def dropTable(self, tableName) :
        """ 
        is used to drop / delete table
        * tableName : String 
        """
        try :
            sql = 'DROP TABLE ' + tableName;
            self.cursor.execute(sql)
            print(tableName , ' is dropped successfully!')
        except Exception as e :
            print("Error is: " + str(e))
         
        
    def dropColumns(self, tableName, columnNames) :
        """
        Is used to delete column/s :
        * tableName   : The table name (string).
        * columnNames : Enter the column/s name (list).
        """
        try :
            for x in columnNames :
                sql = 'ALTER TABLE '+ tableName + ' DROP COLUMN ' + x
                self.cursor.execute(sql)
                print(x , 'dropped successfully!')
        except Exception as e :
            print("Error is: " + str(e))   
            
            
            
    def alterColumn(self, tableName, oldColumnName, newColumnName='', constarint = '', dataType = None) :
        
        """
        It is used to modify column data_type , add constraint.
        it takes 5 args :
        * tableName. 
        * oldColumnName. 
        * newColumnName(if you want to change it).
        * constraint (if you want also ).
        * dataType.
        
        """
        
        try :
            if dataType != None :
                sql = 'alter table ' + tableName + ' modify column ' + oldColumnName +" "+  newColumnName + " " +  dataType + " " + constarint
                self.cursor.execute(sql)
            print("Column is altered successfully!")    
        except Exception as e :
            print("Error is: " + str(e))
            
    
    def insert(self, tableName, columnNames, values ) :
        """
        To add new row to the table 
        * table_name   : The name of the table. 
        * column_names : The column you want to use.
        * values       : List of the given values and it should be 6 in length. 
        
        """ 
        x = ['%s'] * (len(values))    
        m = ','.join(x)
        if len(values) > 1 :
            columnNames = ','.join(columnNames)
        sql = 'insert into ' + tableName + ' (' + columnNames + ') values (' + m + ' )' 
        try :
            self.cursor.execute(sql, values)
            self.DB.commit()
            print("row added!")
        except Exception as e:
            print("Error is: " + str(e))
            
            
    def readAll(self, tableName) :
        "To read all data "
        
        sql = 'select* from ' + tableName
        try :
            self.cursor.execute(sql)
            row = self.cursor.fetchall()
            return row
        except Exception as e :
            print("Error is: " + str(e))
            
    def read(self, tableName, key, columns, conditions) :
        """
        Read the row that match the key and condition
        * tableName : The name of the table
        * key       : Which column you want
        * columns   : Related to conditions
        * condition : Where the key equal to the condition
         **output   : Return row/s that match/es the condition
        
        """
        if columns == '' :
            wh = '%s'
        else :    
            wh = columns[0] + ' = %s'
            if len(columns) > 1 :  
                for i in range(1 , len(columns)) :
                    wh += ' and '
                    wh += columns[i] + ' = %s'
        
        sql = 'SELECT '+ str(key) + ' FROM '+ tableName + ' WHERE ( '+ wh+ ' )'
        try :
            self.cursor.execute(sql ,conditions)
            row = self.cursor.fetchall()
            return row
        except Exception as e :
            print('Error is: ' + str(e))
            
            
    def deleteRow(self, tableName, key, condition) :
        sql = 'DELETE FROM ' + tableName + ' WHERE ' + key +' = %s' 
        try :
            self.cursor.execute(sql , (condition ,))
            self.db.commit()
            print("row deleted!")
        except Exception as e :
            print("Error is: " + str(e))
    
    def deleteAll(self, tableName) :
        "Clear all the table"
        sql = 'DELETE * FROM ' + tableName
        try :
            self.cursor.execute(sql)
            print("Data is cleared!")
            
        except Exception as e :
            print("Error is: " + str(e))
            
            
    def close(self):
        """
        Close the database connection.
        """
        self.DB.close()
   
    

In [6]:
#create object of the database 

obj = database('training-data')


In [20]:
parent = 'D:\\Graduation Project'
mainDir = 'AD - Test Data'
pathMainDir = Path(parent+ '//' + mainDir)

# Iterate through dirs into the main dir 
for sub in pathMainDir.iterdir():
    obj.insert('dir' , ('main_dir', 'sub_dir') , [mainDir , sub.name])
    
# Get the subs-dir of the main dir  
subDir = obj.read('dir', ('sub_dir'), ['main_dir'], mainDir)



#2 - get the list of the data (ids of patients)
for sub in subDir :
    p = Path(parent + '\\' + mainDir + '\\' + sub[0])
    #get_subDir = obj.read('dir' , ('id') , ['sub_dir'] , sub[0])[0][0]
    for _id in p.iterdir() :
        if _id.is_dir():
            created_at = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
            obj.insert('users' , ('patient_id' , 'created_at') , [_id.name , created_at])
            

Error is: (1062, "Duplicate entry 'MRI' for key 'sub_dir'")
Error is: (1062, "Duplicate entry '011_S_0003' for key 'patient-id'")
Error is: (1062, "Duplicate entry '021_S_0984' for key 'patient-id'")
Error is: (1062, "Duplicate entry '041_S_0898' for key 'patient-id'")
Error is: (1062, "Duplicate entry '100_S_0006' for key 'patient-id'")
Error is: (1062, "Duplicate entry '128_S_0230' for key 'patient-id'")


In [21]:



def fillData(*args):
    """
    It takes any number of strins and convert it to list. 
    """
    result = list()
    for item in args :
        result.append(item)
    return result


def convertToString(l) :
    return ' '.join(l) 


def convertDate(d) :
    import datetime
    """ 
    Convert the data from the form (%y-%m-%d) to (%y).
    """
    x = d.split('_')[0]
    x = datetime.datetime.strptime(x, '%Y-%m-%d')
    x = x.strftime('%Y')
    return x 



# Select all dirs and loop through them 
allDirs = obj.read('dir', ('main_dir , sub_dir'), '' , [1])

for _dir in allDirs :
    
    udir= '\\'.join(list(_dir))
    
    path = Path(parent + '\\' + udir) 
    getIdSub = obj.read('dir', ('id'), ['main_dir' , 'sub_dir'], list(_dir))[0][0]
  
    for _id in path.iterdir() :
        path_name = parent + '\\' + udir+ '\\' + _id.name  
        path = Path(path_name)
        getIdPatient = obj.read('users', ('id'), ('patient_id',), [_id.name])[0][0]
        
        for ray in path.iterdir() :
            obj.insert('ray', ('name') ,[ray.name])
            getRayId = obj.read('ray', ('id'), ('name',), [ray.name])[0][0]
           
            for datetime in ray.iterdir():
                for num in datetime.iterdir():
                    
                    obj.insert('number', ('name'), [num.name])
                    getNumId = obj.read('number', ('id'), ('name',), [num.name])[0][0]
                
                    for img in num.iterdir() :
                        if img.glob('**/*.dcm') :
                            photoName = img.name
                            created_at = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
                        
                            data = fillData(getIdSub, getIdPatient, getRayId, datetime.name, getNumId, photoName ,created_at)
                        
                        obj.insert('photo', ('dir_id', 'user_id', 'ray_id', 'examdate', 'number_id', 'photo_name', 'created_at'), data)
                        
                        #Get the photo id where the name of image is matched from photo table 
                        getPhotoId = obj.read('photo', ('id'), ('photo_name',), [photoName])[0][0]
                        
                        #Convert the form of the date. 
                        date = convertDate(str(datetime.name))
                        
                        #get diagnosis from patient table
                        print(_id.name , date)
                        try :
                            
                            getDiagnosis = obj.read('patient', ('diagnosis'), ('pt_id', 'examdate'), [_id.name , date])[0][0]
                        except Exception as e :
                            print("Error is: " + str(e))
                        #Get the id of diagnosis from diagnosis. 
                        getDiagnosisId = obj.read('diagnosis', ('id'), ('name',), [getDiagnosis])[0][0]
                        
                        #Insert into scans table id of photo and the diagnosis of the photo. 
                        obj.insert('scans', ('photo_id', 'diagnosis_id'), [getPhotoId, getDiagnosisId]) 

row added!
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
r

row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
r

row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
row added!
011_S_0003 2006
row added!
r

row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
r

row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
011_S_0003 2007
row added!
row added!
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
row added!
row added!
011_S_0003 2005
r

...

row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tup

row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
row added!
100_S_0006 2009
E

row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index ou

row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
100_S_0006 2007
row added!
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index out of range
row added!
row added!
100_S_0006 2009
Error is: tuple index ou

row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
128_S_0230 2006
row added!
row added!
1

In [21]:
# Select all ids. 
allIDs = obj.read('users', 'patient_id','',[1] )

# Print patient_ids.
for i,_id in zip(range(len(allIDs)), allIDs) :
    print(i , ' - ' , _id[0])
    
idNumber = len(allIDs)
while idNumber not in range(len(allIDs)) :
    idNumber = int(input("Choose the id of patient you want "))
    print("Wrong number!")

getIdOfPatient = obj.read('users', ('id'),('patient_id',), [allIDs[idNumber][0]] )

# Select all rays.
idRays = list(set(obj.read('photo' , ('ray_id') , ('user_id',) ,getIdOfPatient[0][0])))

# Print ray names. 
for i in range(len(idRays)):    
    print(str(i) + ' - ' + obj.read('ray' ,('name') , ('id',) , idRays[i][0])[0][0])
    
rayNumber = len(idRays)
while rayNumber not in range(len(idRays)):
    rayNumber = int(input('choose the type of the ray you want '))    

listOFPhoto = obj.read('photo' , ('photo_name') , ('user_id' , 'ray_id') , [getIdOfPatient , idRays[rayNumber]])

for i in range(len(listOFPhoto)) :
    print(str(i+1) + ' - ' + listOFPhoto[i][0])

0  -  011_S_0003
1  -  021_S_0984
2  -  041_S_0898
3  -  100_S_0006
4  -  128_S_0230
Choose the id of patient you want 5
Wrong number!
Choose the id of patient you want 0
Wrong number!
0 - Axial_PD-T2_TSE
1 - B1-calibration_Body
2 - B1-calibration_Head
3 - Localizer
4 - MPRAGE
5 - MPRAGE_Repeat
choose the type of the ray you want 6
choose the type of the ray you want 5
1 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20050901114842183_1_S9127_I7054.dcm
2 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20050901114842714_10_S9127_I7054.dcm
3 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20050901114843011_100_S9127_I7054.dcm
4 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20050901114843324_101_S9127_I7054.dcm
5 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20050901114843621_102_S9127_I7054.dcm
6 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20050901114843933_103_S9127_I7054.dcm
7 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20050901114844230_104_S9127_I7054.dcm
8 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20050

431 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085234748_54_S39295_I73249.dcm
432 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085234904_55_S39295_I73249.dcm
433 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085235060_56_S39295_I73249.dcm
434 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085235232_57_S39295_I73249.dcm
435 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085235388_58_S39295_I73249.dcm
436 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085235544_59_S39295_I73249.dcm
437 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085235716_6_S39295_I73249.dcm
438 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085235873_60_S39295_I73249.dcm
439 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085236029_61_S39295_I73249.dcm
440 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085236185_62_S39295_I73249.dcm
441 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914085236513_63_S39295_I73249.dcm
442 - ADNI_011_S_0003_MR_MPRAGE_Repeat__br_raw_20070914